In [ ]:
# na te zajęcia potrzebować będziemy wersji lokalnej SPARKA (może być docker !)
# git clone 
# docker build -t spark .
# docker run -p 8888:8888 spark


from pyspark.sql import SparkSession
from pyspark.streaming import StreamingContext

# Create a local StreamingContext with two working thread
# and batch interval of 1 second
spark = SparkSession\
        .builder\
        .master("local[2]") \ # TUTAJ WAŻNE ABY BYŁY PRZYNAJMNIEJ DWA PROCESORY
        .appName("Stream_Socket")\
        .getOrCreate()


sc = spark.sparkContext

In [ ]:
# biblioteka re - regular expression 
import re
# =================
# RDD - czyli podstawowy obiekt do obsługi danych dla SPARKA
rdd = sc.parallelize([1,2,3])
rdd.take(2)

In [ ]:
# ===================
# Prosty Word Count 
# r'[a-z'] - oznacza jeden znak dla liter od a do z (male) + apostrof
# []+ - oznacza, że może pojawić się jeden lub więcej znaków
# pamiętajcie, że przetwarzamy x.lower()
# Licznik ten jest wygenerowany na podstawie map-reduce
# ===================

sc.textFile("RDD_input") \
.map(lambda x: re.findall(r"[a-z']+", x.lower())) \
.flatMap(lambda x: [(y, 1) for y in x]) \
.reduceByKey(lambda x,y: x + y) \
.collect()

## SPARK STREAMING

Część Sparka odpowiedzialna za przetwarzanie danych w czasie rzeczywistym. 


<img src="https://spark.apache.org/docs/latest/img/streaming-arch.png"/>

Dane mogą pochodzić z różnych źródeł np. sokety TCP, Kafka, etc. 
Korzystając z poznanych już metod `map, reduce, join, oraz window` można w łatwy sposób generować przetwarzanie strumienia tak jaby był to nieskończony ciąg RDD. 
Ponadto nie ma problemu aby wywołać na strumieniu operacje ML czy wykresy. 

Cała procedura przedstawia się następująco: 

<img src="https://spark.apache.org/docs/latest/img/streaming-flow.png"/>

SPARK STREAMING w tej wersji wprowadza abstrakcje zwaną `discretized stream` *DStream* (reprezentuje sekwencję RDD).

Operacje na DStream można wykonywać w API JAVA, SCALA, Python, R (nie wszystkie możliwości są dostępne dla Pythona). 

## Spark Streaming potrzebuje minium 2 rdzenie !

----
- **StreamingContext(sparkContext, batchDuration)** - reprezentuje połączenie z klastrem i służy do tworzenia DStreamów, `batchDuration` wskazuje na granularność batch'y (w sekundach)
- **socketTextStream(hostname, port)** - tworzy DStream na podstawie danych napływających ze wskazanego źródła TCP
- **flatMap(f), map(f), reduceByKey(f)** - działają analogicznie jak w przypadku RDD z tym że tworzą nowe DStream'y
- **pprint(n)** - printuje pierwsze `n` (domyślnie 10) elementów z każdego RDD wygenerowanego w DStream'ie
- **StreamingContext.start()** - rozpoczyna działania na strumieniach
- **StreamingContext.awaitTermination(timeout)** - oczekuje na zakończenie działań na strumieniach
- **StreamingContext.stop(stopSparkContext, stopGraceFully)** - kończy działania na strumieniach

Obiekt StreamingContext można wygenerować za pomocą obiektu SparkContext.

In [ ]:
ssc = StreamingContext(sc, 5) # ustawiłem na 5 sekund aby był czas na wpisywanie

Po wygenerowaniu obiektu `ssc` musisz wskazać źródło i utworzyć na jego podstawie DStream. Określić wszystkie transformacje. Uruchomić metodę `start()`, która powoduje nasłuchiwanie. Włączyć oczekiwanie na zakończenie procesu `awaitTermination()` bądź zatrzymać nasłuch ręcznie `stop()`. 

- po rozpoczęciu nasłuchu nie można już ustawić nowych przekształceń !
- po zatrzymaniu nie można zrestartować
- tylko jeden StreamingContext aktywny na JVM 

In [ ]:
# DStream - dane pobierane z socketu TCP - na porcie 9999

lines = ssc.socketTextStream("localhost", 9999)

In [ ]:
words = lines.flatMap(lambda x: re.findall(r"[a-z']+", x.lower()))
wordCounts = words.map(lambda word: (word,1)).reduceByKey(lambda x,y: x+y)
wordCounts.pprint()

In [ ]:
ssc.start()             # Start the computation
ssc.awaitTermination()  # Wait for the computation to terminate
ssc.stop(True,True)

In [ ]:
# pierwsza wersja nadawania na TCP 9999 (unix)
# w konsoli linuxowej netcat Nmap for windows
!nc -lk 9999

# wpisujesz tekst
# jeśli się nie uda to generuj plik poniżej (Możecie też zmienić plik źródłowy)
# zweryfikujcie żeby ścieżka była odpowiednia

In [ ]:
%%file stream.py

from socket import *
import time

rdd = list()
with open("RDD_input", 'r') as ad:
    for line in ad:
        rdd.append(line)

HOST = 'localhost'
PORT = 9999
ADDR = (HOST, PORT)
tcpSock = socket(AF_INET, SOCK_STREAM)
tcpSock.bind(ADDR)
tcpSock.listen(5)


while True:
    c, addr = tcpSock.accept()
    print('got connection')
    for line in rdd:
        try:
            c.send(line.encode())
            time.sleep(1)
        except:
            break
    c.close()
    print('disconnected')

In [ ]:
# w osobnej konsoli (możesz uruchomić ją w jupyter notebook)
! python stream.py 

<img src="https://spark.apache.org/docs/latest/img/streaming-dstream.png"/>

<img src="https://spark.apache.org/docs/latest/img/streaming-dstream-ops.png"/>

In [ ]:
## dane do strumienia można wygenerować też jako listę i wykorzystać tzw kolejkę 
## kolejka to tak jak w sklepie obsługujemy pierwszego (ostatni dochodzi na koniec kolejki)
## po obsłużeniu wypada pierwszy i kolejka się zmniejsza

In [ ]:
from pyspark.sql import SparkSession
from pyspark.streaming import StreamingContext

spark = SparkSession\
        .builder\
        .master("local[2]") \ # TUTAJ WAŻNE ABY BYŁY PRZYNAJMNIEJ DWA PROCESORY
        .appName("Stream_Kolejka")\
        .getOrCreate()
sc = spark.sparkContext
ssc = StreamingContext(sc, 1) # ustawiłem na 5 sekund aby był czas na wpisywanie

# 10 pakietów po 1000 liczb
rddQueue = []
for i in range(10):
        rddQueue += [sc.parallelize(
            [j for j in range(1, 1001)], 10)]

In [ ]:
inputStream = ssc.queueStream(rddQueue)

mappedStream = inputStream.map(lambda x: (x % 10, 1))
reducedStream = mappedStream.reduceByKey(lambda a, b: a + b)

reducedStream.pprint()

In [ ]:
# tym razem trzeba trochę inaczej uruchomić i wprowadzić czas między
import time
ssc.start()
time.sleep(10)
ssc.stop(stopSparkContext=True, stopGraceFully=True)

## Stateful Wordcount 

Operacja `updateStateByKey` pozwala łączyć ze sobą wyniki otrzymywane na poszczególbych DStreamach. Dzięki tej operacji możesz w sposób ciągły uzupełniać informacje !

Aby Spark Streaming mógł łączyć dane z wielu batchy (stateful transformations) konieczne jest wskazanie lokalizacji gdzie zapisywane będą checkpointy.

1. Zdefiniuj stan podstawowy
2. wskaż funkcję łączącą 

----
- **checkpoint(directory)** - wskazuje gdzie zapisywane będą checkpointy z operacji na DStream'ach
- **updateStateByKey(updateFunc)** - zwraca nowy DStream zawierający informację o bieżącym stanie poszczególnych kluczy, stan każdego klucza odświeżany jest przy pomocy `updateFunc`

In [ ]:
def updateFunc(newValues, runningCount):
    if runningCount is None:
        runningCount = 0
    return sum(newValues, runningCount)

In [ ]:
from pyspark.sql import SparkSession
from pyspark.streaming import StreamingContext

spark = SparkSession\
        .builder\
        .master("local[2]") \ # TUTAJ WAŻNE ABY BYŁY PRZYNAJMNIEJ DWA PROCESORY
        .appName("Stream_stateful")\
        .getOrCreate()

sc = spark.sparkContext
ssc = StreamingContext(sc, 1)

In [ ]:
# tutaj zdecydujcie czy puszczacie z pliku czy z konsoli (z konsoli dobrze widac jak dziala)
lines = ssc.socketTextStream("localhost", 9999)

words = lines.flatMap(lambda x: re.findall(r"[a-z']+", x.lower()))
pairs = words.map(lambda word: (word, 1))

runningCounts = pairs.updateStateByKey(updateFunc)

runningCounts.pprint()

In [ ]:
ssc.start()
ssc.awaitTermination()

ssc.stop(True,True)

## Redukcja w oknach 

----
- **reduceByKeyAndWindow(func, invFunc, windowDuration, slideDuration)** - zwraca nowy DStream powstały w wyniku stosowania przyrostowo reduceByKey wewnątrz zdefiniowanego okna. Zredukowane wartości dla nowego okna obliczane są z wykorzystaniem wartości starego okna poprzez: 
1. zredukowanie (dodanie) nowych wartości, 
2. "odwrotne zredukowanie" (odjęcie) wartości które opuściły już okno

In [ ]:
from pyspark.sql import SparkSession
from pyspark.streaming import StreamingContext

spark = SparkSession\
        .builder\
        .master("local[2]") \ # TUTAJ WAŻNE ABY BYŁY PRZYNAJMNIEJ DWA PROCESORY
        .appName("Stream_windows")\
        .getOrCreate()

sc = spark.sparkContext
ssc = StreamingContext(sc, 2)
ssc.checkpoint("tmp")

In [ ]:
lines = ssc.socketTextStream("localhost", 9999)
words = lines.flatMap(lambda x: re.findall(r"[a-z']+", x.lower()))
pairs = words.map(lambda word: (word,1))
# window length - długość trwania okna
# sliding interval - czas w którym wykonywana jest funkcja okna 
windowedWordCounts = pairs.reduceByKeyAndWindow(lambda x, y: x + y, lambda x, y: x - y, 30, 10)

windowedWordCounts.pprint()

In [ ]:
ssc.start()
ssc.awaitTermination()
ssc.stop(True,True)

### Stream do DataFrame

lines = DataFrame reprezentujący nieograniczoną tabelę zawierającą dane strumieniowe. 
Zawiera ona jedną kolumnę o nazwie `value`. Każda nowa linia to wiersz w tabeli. 

<img src="https://spark.apache.org/docs/latest/img/structured-streaming-stream-as-a-table.png"/>

## SPARK Structured Streaming

----
- **SparkSession.readStream.format(source).option(key, value).load()** - tworzy strumieniowy DataFrame
- **DataFrame.writeStream.outputMode(mode).format(source).option(key, value).start()** - wysyła dane ze strumieniowego DataFrame'u "na zewnątrz"; `complete` mode - outputem jest cała zaktualizowana tabela, `append` mode - outputem są jedynie nowe wiersze, `update` mode - outputem są jedynie zaktualizowane wiersze

In [ ]:
%%file spark2.py

from pyspark.sql import SparkSession
import pyspark.sql.functions as f

if __name__ == "__main__":
    spark = SparkSession\
        .builder\
        .master("local[2]") \
        .appName("Stream_DF")\
        .getOrCreate()
    print("="*50)
    print("Zaczynamy DataFrame")
    print("="*50)
    spark.sparkContext.setLogLevel("ERROR")
    lines = spark.readStream\
        .format("socket")\
        .option("host", "localhost")\
        .option("port", 9999)\
        .load()
    words = lines.select(f.explode(f.split(lines.value, " ")).alias("word"))
    wordCounts = words.groupBy("word").count()
    query = wordCounts.writeStream.outputMode("complete").format("console").start()
    query.awaitTermination()
    query.stop()

In [ ]:
! spark-submit spark2.py